# Reproducing Heatmap Experiments
The goal of this notebook is to make it very easy for reviewers and readers of the accompanying paper to reproduce experimental results and make minor modifications, in order to convince themselves the experimental methods are sound, the results valid, and the conclusions appropriate.

In particular, this file contains the code to build the heatmaps seen in figures TODO and their corresponding isolines. Familiarity with the supplied repo is recommended: there are two tutorial notebooks that serve as a primer. 

Recall that the goal of this experiment is to show two (equivalent) statements:

_Amongst the set of combination threat models with the same adversarial strength, the threat model that produces adversarial examples of minimal perceptual distance (according to quantifiable perceptual metrics) utilizes both additive and flow attacks_ 

And the 'dual' of the above statment:

_Amongst the set of combination threat models that produce adversarial examples of equivalent perceptual distance (according to quantifiable perceptual metrics), the threat model that produces attacks most likely to fool a classifier is a combination of both additive and flow attacks_

## Contents
- Preliminaries 
- Building Heatmaps

# Preliminaries 
Setting up models, attacks, etc:

In [ ]:
# EXTERNAL LIBRARY IMPORTS
import numpy as np 
import scipy 
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

import torch # Need torch version 0.3 or 0.4
import torch.nn as nn 
import torch.optim as optim 
assert torch.__version__[:3] in ['0.3', '0.4', '0.5']

use_gpu = torch.cuda.is_available() # STRONGLY RECOMMENDED THAT YOU RUN CODE ON GPU MACHINE
print("Using GPU? " + str(use_gpu))

In [ ]:
# IMPORT BLOCK
# (here we do things so relative imports work )
# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


import config

import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_attacks as aa
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap 
import adversarial_attacks_refactor as aar
import spatial_transformers as st
import prebuilt_attacks as pa 
import utils.experiment_utils as eu


In [ ]:
""" Initialize the dataLoader, classifier, and normalizer """
use_gpu = torch.cuda.is_available()
classifier_net = cifar_loader.load_pretrained_cifar_resnet(flavor=32,
                                                           use_gpu=use_gpu)
EXPERIMENT_NAME = 'linfStadv'
defended_state_dict_file = checkpoints.params_to_filename(EXPERIMENT_NAME, 'resnet32')[-1]
print "Loading %s params into resnet32" % defended_state_dict_file 
classifier_net = checkpoints.load_state_dict_from_filename(defended_state_dict_file, classifier_net)    
classifier_net.eval()

train_loader = cifar_loader.load_cifar_data('train', normalize=False, 
                                            batch_size=128, use_gpu=use_gpu)
val_loader = cifar_loader.load_cifar_data('val', normalize=False, 
                                          batch_size=32, use_gpu=use_gpu, 
                                           shuffle=False)
cifar_normer = utils.DifferentiableNormalize(mean=config.CIFAR10_MEANS,
                                             std=config.CIFAR10_STDS)
examples, labels = next(iter(val_loader))

# Building Heatmaps

In [ ]:
""" Code to build a heatmap as a dictionary mapping (row, col) indices to 

"""

def create_heatmap_v2(classifier_net, discretization=5, stadv_limit=1.0/64, linf_limit=8.0/255.0,
                   num_minibatches=10):
    """
    Arguments:
        classifier_net: the network to attack
        discretization: how many equally spaced intervals between the limits should we try?
        stadv_limit: maximum allowable stadv perturbation
        linf_limit: maximum allowable linf_perturbation
    """
    stadv_inc = stadv_limit / discretization
    linf_inc = linf_limit / discretization
    data_out = []
    attack_ensemble = {} 
    for i in range(discretization + 1):
        for j in range(discretization + 1):
            current_stadv_limit = stadv_inc * (i)
            current_linf_limit = linf_inc * (j)
            attack_ensemble[(i, j)] = pa.build_delta_stadv_pgd(classifier_net, cifar_normer, 
                                                               delta_bound=current_linf_limit, 
                                                               flow_bound=current_stadv_limit, 
                                                               output='eval')
    eval_obj = adveval.AdversarialEvaluation(classifier_net, cifar_normer, use_gpu=use_gpu)
    eval_output = eval_obj.evaluate_ensemble(val_loader, attack_ensemble, verbose=True, 
                                             num_minibatches=num_minibatches)
    
    return eval_output

In [ ]:
""" Code to display heatmap """
def display_heatmap(eval_output, val_to_show, flow_bound, linf_bound, discretization, normalize=False,
                    levels=None, scale=(10,10), no_title=False):
    """ Plots the heatmap     
    """
    assert val_to_show in ['lpips', 'strength']
    preprocess = lambda x: x
    if val_to_show == 'strength':
        val_to_show = 'top1'
        #preprocess = lambda x: 1 - x
        
    disc = discretization    
    heatmap = np.zeros((disc + 1, disc + 1))
    for k, val in eval_output.items():
        if k == 'ground':
            continue 
        i, j = k
        heatmap[i][j] = preprocess(val.results[val_to_show].avg)
        
    vmax = 1.0
    if val_to_show == 'lpips':        
        heatmap = (torch.Tensor(heatmap) * 1000).numpy()
        vmax = np.max(heatmap)
        plot_title = "LPIPS Distance"
    else:    
        plot_title = "Attack Strength"
    if normalize:
        vmax = 1.0
        heatmap = heatmap / np.max(heatmap)  
        plot_title = "(normalized) " + plot_title
        
    plt.figure(figsize=scale, dpi=80, facecolor='w', edgecolor='k')    
    
    if not no_title:
        plt.title(plot_title, fontsize=32)
    plt.xlabel(r"Delta bound", fontsize=32)
    plt.ylabel(r"Flow bound ($\times 64$)", fontsize=32)
    
    to_show_delta = linf_bound * 255.0
    to_show_flow = flow_bound * 64
    x = plt.contourf([to_show_delta / disc * _ for _ in range(disc + 1)], 
                 [to_show_flow / disc * _ for _ in range(disc + 1)], 
                 heatmap, vmin=0, vmax=vmax, xunits=2, levels=levels)
    return x
    


## USER INPUT TO RUN EXPERIMENTS:

In [ ]:
"""
First you'll want to set:
- the flow/additive linf bounds
- how fine the mesh is
- how many minibatches you want to evaluate on per mesh point

And then build up the heatmap for all of these meshpoints

"""
FLOW_BOUND = 1.0 / 64 # THESE ARE TYPICALLY CONSIDERED AS X/64.0 FOR 32x32 IMAGES

LINF_BOUND = 8.0 / 255.0 # THESE ARE TYPICALLY CONSIDERED AS X/255.0 FOR 8-BIT RGB IMAGES

DISCRETIZATION = 10 # THERE ARE (discretization + 1)**2 MESHPOINTS

NUM_MINIBATCHES = 10 # ~1/4 OF CIFAR EVAL SET, FOR MORE INSTANT GRATIFICATION


# This next step takes some time, so only do it once
heatmap_eval_output = create_heatmap_v2(classifier_net, 
                                     discretization=DISCRETIZATION,
                                     stadv_limit=FLOW_BOUND, 
                                     linf_limit=LINF_BOUND, 
                                     num_minibatches=NUM_MINIBATCHES)

In [ ]:
""" And with the eval output built, you can display heatmap. 
    OPTIONS TO TOGGLE HERE: 
    
    val_to_show: ['strength', 'lpips'] : does the heatmap represent strength or LPIPS distance 
    normalize : [True, False] : do we normalize the values such that the max value such that it is 1.0?    
"""

VAL_TO_SHOW = 'strength' # must be one of 'strength', 'lpips'

normalize = (VAL_TO_SHOW == 'lpips') # normalize only lpips values by default

x = display_heatmap(heatmap_eval_output, VAL_TO_SHOW, FLOW_BOUND, LINF_BOUND, DISCRETIZATION, normalize=normalize,
                    levels = [0.05 * i for i in xrange(22)])
y = display_heatmap(heatmap_eval_output, 'lpips', FLOW_BOUND, LINF_BOUND, DISCRETIZATION, normalize=False,
                    levels=[0.1 * i for i in xrange(50)])

#x.collections[1].get_paths()[0].vertices

## A note on interpreting these plots
Recall the statements we wish to prop up with experimental data:

_Amongst the set of combination threat models with the same adversarial strength, the threat model that produces adversarial examples of minimal perceptual distance (according to quantifiable perceptual metrics) utilizes both additive and flow attacks_ 

and 

_Amongst the set of combination threat models that produce adversarial examples of equivalent perceptual distance (according to quantifiable perceptual metrics), the threat model that produces attacks most likely to fool a classifier is a combination of both additive and flow attacks_

If these statements were true, then one would expect the plots of the isolines of the LPIPS plot to be 'more concave' than the isolines of the strength plot. Verify that this is occurring.